In [66]:
import os
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Activation, merging, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import Image

In [41]:
#フォルダ名をクラス名にする フォルダ番号が答え
path = "./sns/train"
folders = os.listdir(path)
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)
answer = {"0":"facebook", "1":"insta","2":"line", "3":"tiktk","4":"twitter","5":"twitter_t"}
print(classes)
print("n_classes =", n_classes)
print(f"{answer = }")

['facebook', 'line', 'tiktok', 'twitter', 'twitter_t']
n_classes = 5
answer = {'0': 'facebook', '1': 'line', '2': 'tiktk', '3': 'twitter', '4': 'twitter_t'}


In [44]:
#訓練データ作成。画像の読み込みとグレースケール化。フォルダ番号が答え
X =[]
y=[]
for label, class_name in enumerate(classes):
    files= glob("./sns/train/" + class_name+"/*.*")
    for file in files:
        img = Image.open(file)
        img = img.convert("L").convert("RGB")
        img = img.resize((224,224))
        img =  np.asarray(img, np.uint8)
        X.append(img)
        y.append(label)
print(len(X))
print(len(y))
print(X[0].shape)
#ndarray化
X_ = np.array(X)
X_ = X_.astype("float32")
X_train = X_/255.0 #正規化
y_ = np.array(y)
y_train = np_utils.to_categorical(y_,n_classes)

195
195
(224, 224, 3)


In [45]:
#testデータ作成　画像の読み込みとグレースケール化
X1 =[]
y1=[]
for label, class_name in enumerate(classes):
    files= glob("./sns/validation/" + class_name+"/*.*")
    for file in files:
        img = Image.open(file)
        img = img.convert("L").convert("RGB")
        img = img.resize((224,224))
        img = np.asarray(img, np.uint8)
        X1.append(img)
        y1.append(label)
print(len(X1))
print(len(y1))
print(X1[0].shape)
X_test_ = np.array(X1)
X_test_ = X_test_.astype("float32")
X_test = X_test_/255.0
y_test_ = np.array(y1)
y_test = np_utils.to_categorical(y_test_,n_classes)
print(len(X_test))
print(len(y_test))
X_test[0].shape

147
147
(224, 224, 3)


In [47]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(195, 224, 224, 3)
(195, 5)
(147, 224, 224, 3)
(147, 5)


In [48]:
#vgg16
input_tensor = Input(shape=(224,224,3))
base_model = VGG16(weights="imagenet", input_tensor=input_tensor, include_top=False)

In [49]:
#後付
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation="softmax"))

In [50]:
#層結合
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [52]:
#学習させない層　15層まで学習トレーニングしない
print('# layers = ', len(model.layers))
for layer in model.layers[:15]:
    layer.trainable = False

# layers =  20


In [53]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [54]:
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
20/20 [==============================] - 60s 39ms/step - loss: 1.5648 - accuracy: 0.5385
Epoch 2/20
20/20 [==============================] - 1s 35ms/step - loss: 0.3486 - accuracy: 0.9282
Epoch 3/20
20/20 [==============================] - 1s 35ms/step - loss: 0.0212 - accuracy: 0.9949
Epoch 4/20
20/20 [==============================] - 1s 35ms/step - loss: 0.0231 - accuracy: 0.9949
Epoch 5/20
20/20 [==============================] - 1s 36ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 6/20
20/20 [==============================] - 1s 35ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 7/20
20/20 [==============================] - 1s 35ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 8/20
20/20 [==============================] - 1s 35ms/step - loss: 4.0755e-05 - accuracy: 1.0000
Epoch 9/20
20/20 [==============================] - 1s 34ms/step - loss: 1.1535 - accuracy: 0.8923
Epoch 10/20
20/20 [==============================] - 1s 36ms/step - loss: 1.6153 - accuracy: 0.9385
Epoc

In [55]:
score = model.evaluate(X_test, y_test, batch_size=20)

8/8 [==============================] - 1s 54ms/step - loss: 0.6280 - accuracy: 0.9864


In [60]:
score[1]

0.9863945841789246

In [ ]:
import pickle
#クラス名の保存
pickle.dump(classes, open("classes.sav","wb"))
#モデルの保存
model.save("./gray.h5")

In [ ]:
from keras.models import load_model
import pickle
import cv2

In [ ]:
images = [image for image in glob("./sns/testyou/*.*")]
images[3]

In [ ]:
import re

In [ ]:
numbers = [re.search(r"\\(.*)",number).group(1) for number in images]
numbers[33]

In [ ]:
results = []
numbers = []

for image in images:
    img = Image.open(image)
    img = img.convert("L").convert("RGB")
    img = img.resize((224,224))
    img = np.asarray(img, np.uint8)
    img = img.astype("float32")
    img /= 255.0
    img = img[None, ...]
    results.append(model.predict(img))
    numbers.append(re.search(r"u\\(.*.jpg)",image).group(1))
len(results)

In [ ]:
#予測収納
answers = []
for result in results:
    np.set_printoptions(precision=3, suppress=True)
    a = result * 100
    answers.append(a)
len(answers)

In [ ]:
for a,name in zip(answers, numbers):
    k = a.argmax()
    # print(name,k)
    print(name, answer[str(k)])

In [ ]:
#1枚テスト
img = cv2.imread("./sns/testyou/tk50.jpg")
img = cv2.resize(img, dsize=(224,224))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]
result = model.predict(img)

In [ ]:
np.set_printoptions(precision=3, suppress=True)
result * 100

In [ ]:
a = result.argmax()
answer[str(a)]

In [65]:
root = glob("./sns/train/*/*.*")
print(len(root))

406


In [ ]:
for i, name in enumerate(glob("./sns/train/facebook/*.*")):
    os.rename(name, f"{name + str(i)}+ .jpg")
